# TFM - Big Data Analytics

## Generación datos sintéticos - Experimento 3 (Clorofila A)

### Virginia Casino Sánchez (vcassan@disca.upv.es)

# Librerías #

In [ ]:
%%time

import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
#os.environ["CUDA_LAUNCH_BLOCKING"]="1"
import sys
import pathlib
from pathlib import Path
import glob
from glob import glob
import math
from math import sqrt, log, prod
import collections
from collections import OrderedDict
import pickle
from pickle import dump
import warnings

try:
    import numpy as np; print('NumPy version: ', np.__version__)
    from numpy import array, asarray, delete, append, concatenate
except:
    !{sys.executable} -m pip install numpy==1.19.5
    import numpy as np; print('NumPy version: ', np.__version__)
    from numpy import array, asarray, delete, append, concatenate

try:
    import pandas as pd; print('Pandas version: ', pd.__version__)
    from pandas import read_csv, DataFrame, concat
except:
    !{sys.executable} -m pip install pandas
    import pandas as pd; print('Pandas version: ', pd.__version__)
    from pandas import read_csv, DataFrame, concat

try:
    import matplotlib as mpl; print('MatPlotLib version: ', mpl.__version__)
    from matplotlib import pyplot as plt
    from matplotlib import dates as md
except:
    !{sys.executable} -m pip install matplotlib
    import matplotlib as mpl; print('MatPlotLib version: ', mpl.__version__)
    from matplotlib import pyplot as plt
    from matplotlib import dates as md

try:
    import tensorflow as tf
    import tensorflow.keras as keras
    print('TensorFlow version: ', tf.__version__)
    #print('Keras version: ', tf.keras.__version__)
    from tensorflow.keras import backend as K
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import TimeDistributed, Conv1D, MaxPooling1D, Flatten, LSTM, Dense
    from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
except:
    !{sys.executable} -m pip install tensorflow
    import tensorflow as tf
    import tensorflow.keras as keras
    print('TensorFlow version: ', tf.__version__)
    #print('Keras version: ', tf.keras.__version__)
    from tensorflow.keras import backend as K
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import TimeDistributed, Conv1D, MaxPooling1D, Flatten, LSTM, Dense
    from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

# Biblioteca para Aprendizaje Profundo
try:
    import torch as torch; print('Torch version: ', torch.__version__)
except:
    !{sys.executable} -m pip install torch
    import torch as torch; print('Torch version: ', torch.__version__)

# Biblioteca para Generación de datos sintéticos
try:
    import gretel_synthetics as gs
    from gretel_synthetics.timeseries_dgan.dgan import DGAN
    from gretel_synthetics.timeseries_dgan.config import DGANConfig, OutputType
    from gretel_synthetics.timeseries_dgan.config import Normalization
except:
    !{sys.executable} -m pip install gretel-synthetics==0.18.1
    #  !{sys.executable} -m pip install git+https://github.com/gretelai/gretel-synthetics.git
    import gretel_synthetics as gs
    from gretel_synthetics.timeseries_dgan.dgan import DGAN
    from gretel_synthetics.timeseries_dgan.config import DGANConfig, OutputType
    from gretel_synthetics.timeseries_dgan.config import Normalization

import openpyxl
from openpyxl import Workbook

from scipy.spatial.distance import jensenshannon
from scipy.stats import wasserstein_distance
from scipy.stats import ks_2samp
import matplotlib.ticker as ticker

import time

# Configuraciones
np.set_printoptions(linewidth=1000)
pd.set_option('display.max_columns', None)
torch.set_num_threads(60)
#warnings.filterwarnings('ignore')

print()

# Parámetros a configurar #

In [ ]:
%%time

exp = 3
prueba = 64

look_back = 12

keys = [1,2,10]

FILE = 'data_chla.csv'
key = FILE

In [ ]:
batch_size_min = 32
attribute_noise_dim = 10
feature_noise_dim = 10
attribute_num_layers = 3
attribute_num_units = 100
feature_num_layers = 1
feature_num_units = 100
use_attribute_discriminator = True
normalization = Normalization.ZERO_ONE
apply_feature_scaling = True
apply_example_scaling = True
#binary_encoder_cutoff = 150
forget_bias = False
gradient_penalty_coef = 10.0
attribute_gradient_penalty_coef = 10.0
attribute_loss_coef = 1.0
generator_learning_rate= 0.001
generator_beta1 = 0.5
discriminator_learning_rate = 0.001
discriminator_beta1 = 0.5
attribute_discriminator_learning_rate = 0.001
attribute_discriminator_beta1 = 0.5
epochs = 500
discriminator_rounds = 1
generator_rounds = 1
mixed_precision_training = False

# Métodos #

In [ ]:
%%time

# Mostrar algunas de las muestras de formación de 1 día
def plot_day(real, generated, k, label):
    file_svg_cmp = f'comp_{exp}_{prueba}_synth-{label}.svg'
    file_png_cmp = f'comp_{exp}_{prueba}_synth-{label}.png'

    ancho = 15
    alto = 7
    label_x = 'Fecha'
    label_y = 'Clorofila (mg/m3)'
    titulo = f'Comparativa Experimento {exp} Prueba {prueba}'


    xaxis_1day = dataset.index[0:real.shape[0]]
    # Crea una figura con dos gráficos uno al lado del otro
    fig, ax = plt.subplots(figsize=(ancho, alto)) # Ajusta el tamaño según prefieras

    # Dibuja la primera gráfica
    for i, c in enumerate(dataset.columns):
        plt.plot(xaxis_1day, real[:, i], label=c, color = 'royalblue')
        plt.plot(xaxis_1day, generated[:, i], label=c, color = 'seagreen')

    # axs[0].legend(fontsize=20)
    ax.set_xlabel(label_x, fontsize=30, labelpad=25)
    ax.set_ylabel(label_y, fontsize=30, labelpad=25)
    # axs[0].xaxis.set_major_locator(md.HourLocator(byhour=range(2, 24, 3)))
    # axs[0].xaxis.set_major_formatter(md.DateFormatter("%H:%M"))
    ax.set_title(titulo, fontsize=35, pad=10)

    ax.set_facecolor('white')
    ax.grid(True, color='lightgrey', linestyle='--', linewidth=0.7, axis='y', which='major')
    ax.yaxis.grid(True, color='lightgrey', linestyle='--', linewidth=0.2, which='minor')
    ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(10))
    ax.tick_params(axis='y', labelsize=25)
    ax.tick_params(axis='x', which='major', labelsize=25, rotation= 90)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # xaxis_2day = dataset.index[0:generated.shape[0]]
    # # Dibuja la segunda gráfica (igual a la primera)
    # for i, c in enumerate(dataset.columns):
    #     axs[1].plot(xaxis_2day, generated[:, i], label=c, color = 'seagreen')

    # # axs[1].legend(fontsize=20)
    # axs[1].set_xlabel(label_x, fontsize=25, labelpad=25)
    # axs[1].set_ylabel(label_y, fontsize=25, labelpad=25)
    # axs[1].set_title('Valores generados', fontsize=25, pad=10)

    # axs[1].set_facecolor('white')
    # axs[1].grid(True, color='lightgrey', linestyle='--', linewidth=0.7, axis='y', which='major')
    # axs[1].yaxis.grid(True, color='lightgrey', linestyle='--', linewidth=0.2, which='minor')
    # axs[1].yaxis.set_major_locator(ticker.MultipleLocator(5))
    # axs[1].yaxis.set_minor_locator(ticker.MultipleLocator(10))
    # axs[1].tick_params(axis='y', labelsize=25)
    # axs[1].tick_params(axis='x', which='major', labelsize=25, rotation= 90)
    # axs[1].spines['top'].set_visible(False)
    # axs[1].spines['right'].set_visible(False)

    # # Ajusta la disposición y guarda la figura
    # # plt.tight_layout()
    # # plt.savefig(f"numEx{prueba}-{label}-{key}.jpg", bbox_inches='tight')
    # # plt.show()
    # Ajustar imagen
    plt.tight_layout()

    # Guardar imagen
    plt.savefig(file_svg_cmp, format='svg')
    plt.savefig(file_png_cmp, format='png')

    # Mostrar imagen
    plt.show()

def excel_ejecuciones(datos):
    nombre_archivo = f'ejecuciones_{exp}_{prueba}.xlsx'
    try:
        # Intentar cargar el archivo existente
        libro_excel = openpyxl.load_workbook(nombre_archivo)
        hoja_excel = libro_excel.active
    except FileNotFoundError:
        # Si el archivo no existe, crear uno nuevo
        libro_excel = Workbook()
        hoja_excel = libro_excel.active

        # Añadir encabezados si es la primera vez que se crea el archivo
        encabezados = ['Experimento',
                       'Prueba',
                       'max_sequence_len',
                       'sample_len',
                       'attribute_noise_dim',
                       'feature_noise_dim',
                       'attribute_num_layers',
                       'attribute_num_units',
                       'feature_num_layers',
                       'feature_num_units',
                       'use_attribute_discriminator',
                       'normalization',
                       'apply_feature_scaling',
                       'apply_example_scaling',
                       'forget_bias',
                       'gradient_penalty_coef',
                       'attribute_gradient_penalty_coef',
                       'attribute_loss_coef',
                       'generator_learning_rate',
                       'generator_beta1',
                       'discriminator_learning_rate',
                       'discriminator_beta1',
                       'attribute_discriminator_learning_rate',
                       'attribute_discriminator_beta1',
                       'batch_size',
                       'epochs',
                       'discriminator_rounds',
                       'generator_rounds',
                       'mixed_precision_training',
                       'tiempo',
                       'lookback']
        hoja_excel.append(encabezados)

    # Añadir datos a una nueva fila
    hoja_excel.append(datos)

    # Guardar el archivo
    libro_excel.save(nombre_archivo)

print()

# Leer Datasets #

In [ ]:
%%time
dataset = read_csv(FILE, parse_dates=True, index_col='date', sep=",")

# Modelo de Generación de Datos Sintéticos #

In [ ]:
%%time

inicio = time.time()

features = dataset.to_numpy()

n = features.shape[0] // look_back

features = features[:(n*look_back),:].reshape(-1, look_back, features.shape[1])

# Trazar datos reales de 1 día
print('Datos Reales')
# plot_day(features[1,:,:], key, "real-1")
# plot_day(features[2,:,:], key, "real-2")
# plot_day(features[3,:,:], key, "real-3")

# (# ejemplos, # puntos temporales, # características)
print(f'Tamaño datos reales: {features.shape}')

# Entrenar el modelo DGAN
model = DGAN(DGANConfig(
    max_sequence_len = features.shape[1],
    sample_len = look_back,
    attribute_noise_dim = attribute_noise_dim,
    feature_noise_dim = feature_noise_dim,
    attribute_num_layers = attribute_num_layers,
    attribute_num_units = attribute_num_units,
    feature_num_layers = feature_num_layers,
    feature_num_units = feature_num_units,
    use_attribute_discriminator = use_attribute_discriminator,
    normalization = normalization,
    apply_feature_scaling = apply_feature_scaling,
    apply_example_scaling = apply_example_scaling,
    #binary_encoder_cutoff = binary_encoder_cutoff,
    forget_bias = forget_bias,
    gradient_penalty_coef = gradient_penalty_coef,
    attribute_gradient_penalty_coef = attribute_gradient_penalty_coef,
    attribute_loss_coef = attribute_loss_coef,
    generator_learning_rate = generator_learning_rate,
    generator_beta1 = generator_beta1,
    discriminator_learning_rate = discriminator_learning_rate,
    discriminator_beta1 = discriminator_beta1,
    attribute_discriminator_learning_rate = attribute_discriminator_learning_rate,
    attribute_discriminator_beta1 = attribute_discriminator_beta1,
    batch_size=min(batch_size_min, features.shape[0]),
    epochs=epochs,
    discriminator_rounds = discriminator_rounds,
    generator_rounds = generator_rounds,
    mixed_precision_training=mixed_precision_training
))

model.train_numpy(
    features,
    feature_types=[OutputType.CONTINUOUS] * features.shape[2],
)

start_date = dataset.index.min()
end_date = dataset.index.max()

# Generar datos sintéticos
days_to_generate = len(pd.date_range(start_date, end_date, freq='D'))
_, synthetic_features = model.generate_numpy(days_to_generate)

# Trazar datos sintéticos de 1 día
print('Datos sintéticos')
#plot_day(features[keys[0],:,:],synthetic_features[keys[0],:,:], keys[0], f"{keys[0]}")
#plot_day(features[keys[1],:,:],synthetic_features[keys[1],:,:], keys[1], f"{keys[1]}")
#plot_day(features[keys[2],:,:],synthetic_features[keys[2],:,:], keys[2], f"{keys[2]}")


# REVISAR
# Paso 1: Aplanar las secuencias
flattened_features = synthetic_features.reshape(-1, synthetic_features.shape[2])

# Paso 2: Crear identificadores para secuencia y paso temporal
sequence_indices = np.repeat(range(synthetic_features.shape[0]), synthetic_features.shape[1])
timesteps = np.tile(range(synthetic_features.shape[1]), synthetic_features.shape[0])

# Paso 3: Crear el DataFrame con las columnas de Secuencia, Timestep y Feature
synthetic_df_graphic = pd.DataFrame({
    'sequence': sequence_indices,
    'timestep': timesteps,
    'feature': flattened_features.flatten()
})

tiempo = time.time() - inicio
# Guardar resultados
datos_a_agregar = [exp,
                   prueba,
                    features.shape[1],
                    look_back,
                    attribute_noise_dim,
                    feature_noise_dim,
                    attribute_num_layers,
                    attribute_num_units,
                    feature_num_layers,
                    feature_num_units,
                    use_attribute_discriminator,
                    f"{normalization}",
                    apply_feature_scaling,
                    apply_example_scaling,
                    forget_bias,
                    gradient_penalty_coef,
                    attribute_gradient_penalty_coef,
                    attribute_loss_coef,
                    generator_learning_rate,
                    generator_beta1,
                    discriminator_learning_rate,
                    discriminator_beta1,
                    attribute_discriminator_learning_rate,
                    attribute_discriminator_beta1,
                    min(batch_size_min, features.shape[0]),
                    epochs,
                    discriminator_rounds,
                    generator_rounds,
                    mixed_precision_training,
                    tiempo,
                    look_back]

excel_ejecuciones(datos_a_agregar)
print()

In [ ]:
start_date = dataset.index.min()
end_date = dataset.index.max()
hourly_date_range = pd.date_range(start=start_date, end=end_date, freq="D")

In [ ]:
timesteps_per_sequence = synthetic_df_graphic['sequence'].iloc[-1] + 1
num_sequences = look_back

In [ ]:
# Repeat the hourly range for each sequence and then truncate to match the needed length
hourly_dates = pd.Series(hourly_date_range[:timesteps_per_sequence].repeat(num_sequences))

# Update the synthetic dataframe with these hourly timestamps
synthetic_df_graphic['date'] = hourly_dates.reset_index(drop=True)

In [ ]:
# synthetic_df_graphic['date'] = start_date + pd.to_timedelta(synthetic_df_graphic['timestep'], unit='D')

# # Graficar usando la fecha
# plt.figure(figsize=(12, 6))
# plt.plot(synthetic_df_graphic['date'], synthetic_df_graphic['feature'], color='blue')
# plt.xlabel('Fecha')
# plt.ylabel('Feature')
# plt.title('Serie Temporal Completa con Fechas')
# plt.show()

# Comparar Resultados Reales-Sintéticos #

In [ ]:
# Extraer los valores reales y generados del dataset
real_values = dataset['chla'].dropna().values


def calculate_jsd(real_data, generated_data, bins=50):
    # Obtener el rango común para ambos conjuntos de datos
    data_min = min(real_data.min(), generated_data.min())
    data_max = max(real_data.max(), generated_data.max())

    # Crear bin edges basados en este rango común
    bins = np.linspace(data_min, data_max, 100)  # Cambia 50 por el número de bins que desees

    # Generar histogramas usando los mismos bin edges
    # Histograma de los datos reales y generados
    real_hist, bin_edges = np.histogram(real_data, bins=bins, density=True)
    generated_hist, _ = np.histogram(generated_data, bins=bins, density=True)

    # Normalizar las distribuciones para que sumen 1
    real_hist = real_hist / np.sum(real_hist)
    generated_hist = generated_hist / np.sum(generated_hist)

    # Calcular la distancia de Jensen-Shannon
    jsd = jensenshannon(real_hist, generated_hist, base=2)

    return jsd

def calculate_wd(real_data, generated_data):
  return wasserstein_distance(real_data, generated_data)

def calculate_ks(real_data, generated_data):
  return ks_2samp(real_data, generated_data)

In [ ]:
resultados = []
# Iterar sobre cada timestep y su correspondiente DataFrame
for timestep, data in synthetic_df_graphic.groupby('timestep'):
    print(f"Timestep: {timestep}")
    print()
    generated_values = data['feature'].dropna().values
    # Calcular la distancia de Jensen-Shannon
    jsd_value = calculate_jsd(real_values, generated_values)

    print(f"Distancia de Jensen-Shannon: {jsd_value}")
    print()

    # Calcular la distancia de Wasserstein
    wasserstein_dist = calculate_wd(real_values, generated_values)
    print(f"Wasserstein Distance: {wasserstein_dist}")
    print()

    # Realizar el Kolmogorov-Smirnov Test
    ks_stat, p_value = calculate_ks(real_values, generated_values)
    print(f"K-S Statistic: {ks_stat}")
    print(f"P-Value: {p_value}")
    print()
    print('-----------------------------------------------------')
    # print(data['feature'])
    # Agregar el timestep y el valor calculado a la lista como una tupla
    resultados.append((timestep, jsd_value, wasserstein_dist, ks_stat, p_value))

resultados_df = pd.DataFrame(resultados, columns=['timestep', 'jsd', 'wasserstein', 'ks', 'p-value'])

In [ ]:
# Obtener la fila con el valor más bajo en la columna 'valor_calculado'
mejor_resultado = resultados_df.loc[resultados_df['wasserstein'].idxmin()]

# Mostrar el peor resultado
print("Mejor resultado:")
print(mejor_resultado)

In [ ]:
sub_dataframes = {timestep: data for timestep, data in synthetic_df_graphic.groupby('timestep')}

# Ahora cada timestep tiene su propio DataFrame, al que puedes acceder con sub_dataframes[timestep]
# Ejemplo: Para obtener el DataFrame del timestep 0
df_best_tmsp = sub_dataframes[int(mejor_resultado['timestep'])]

In [ ]:
start_date = dataset.index.min()
end_date = dataset.index.max()
hourly_date_range = pd.date_range(start=start_date, end=end_date, freq="D")

df_best_tmsp["date"]= hourly_date_range

df_best_tmsp = df_best_tmsp.set_index('date')

In [ ]:
ancho = 10
alto = 7
label_x = 'Clorofila (mg/m3)'
label_y = 'Densidad'
titulo = f'Histograma Experimento {exp} Prueba {prueba}'

file_svg_histo = f'histo_{exp}_{prueba}.svg'

file_png_histo = f'histo_{exp}_{prueba}.png'

# Preparación gráfica
fig, ax = plt.subplots(figsize=(ancho, alto))

# Crear el gráfico
plt.hist([dataset['chla'], df_best_tmsp['feature']],
          label=["Real", "Sintético"],
          bins=25,
          density=True, color=['royalblue','seagreen'])
# plt.scatter(dataset.index, dataset['chla'], label='Entrenamiento', color='royalblue', marker='o')
# plt.scatter(df_best_tmsp.index, df_best_tmsp['feature'], label='Generados (reconstrucción)', color='seagreen', marker='x')

# Configuración a los ejes / grid
    # Color de fondo
ax.set_facecolor('white')

    # Configuración del grid
ax.grid(True, color='lightgrey', linestyle='--', linewidth=0.7, axis='y', which='major')
ax.yaxis.grid(True, color='lightgrey', linestyle='--', linewidth=0.2, which='minor')

    # Configuración de los ticks del eje y
ax.yaxis.set_major_locator(ticker.MultipleLocator(10))   # Ticks mayores cada 1
ax.yaxis.set_minor_locator(ticker.MultipleLocator(50)) # Ticks menores cada 0.5

ax.tick_params(axis='y', labelsize=25)

    # Configuración de los ticks del eje x
ax.tick_params(axis='x', which='major', labelsize=25)

# Leyenda
ax.legend(fontsize=20)

# Configuración de las etiquetas
ax.set_xlabel(label_x, fontsize=30, labelpad=25)
ax.set_ylabel(label_y, fontsize=30, labelpad=25)
ax.set_title(titulo, fontsize=35, pad=10)

# Borrar líneas superior y derecha
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar imagen
plt.tight_layout()

# Guardar imagen
plt.savefig(file_svg_histo, format='svg')
plt.savefig(file_png_histo, format='png')

# Mostrar imagen
plt.show()

In [ ]:
ancho = 15
alto = 10
label_x = 'Fecha'
label_y = 'Clorofila (mg/m3)'
titulo = f'Scatter Experimento {exp} Prueba {prueba}'

file_svg_scatter = f'scatter_{exp}_{prueba}.svg'

file_png_scatter = f'scater_{exp}_{prueba}.png'

# Preparación gráfica
fig, ax = plt.subplots(figsize=(ancho, alto))

# Crear el gráfico
plt.scatter(dataset.index, dataset['chla'], label='Entrenamiento', color='royalblue', marker='o')
plt.scatter(df_best_tmsp.index, df_best_tmsp['feature'], label='Generados (reconstrucción)', color='seagreen', marker='x', alpha = 0.4)

# Configuración a los ejes / grid
    # Color de fondo
ax.set_facecolor('white')

    # Configuración del grid
ax.grid(True, color='lightgrey', linestyle='--', linewidth=0.7, axis='y', which='major')
ax.yaxis.grid(True, color='lightgrey', linestyle='--', linewidth=0.2, which='minor')

    # Configuración de los ticks del eje y
ax.yaxis.set_major_locator(ticker.MultipleLocator(10))   # Ticks mayores cada 1
ax.yaxis.set_minor_locator(ticker.MultipleLocator(50)) # Ticks menores cada 0.5

ax.tick_params(axis='y', labelsize=25)

    # Configuración de los ticks del eje x
ax.tick_params(axis='x', which='major', labelsize=25)

# Leyenda
ax.legend(fontsize=20)

# Configuración de las etiquetas
ax.set_xlabel(label_x, fontsize=30, labelpad=25)
ax.set_ylabel(label_y, fontsize=30, labelpad=25)
ax.set_title(titulo, fontsize=35, pad=10)

# Borrar líneas superior y derecha
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar imagen
plt.tight_layout()

# Guardar imagen
plt.savefig(file_svg_scatter, format='svg')
plt.savefig(file_png_scatter, format='png')

# Mostrar imagen
plt.show()

In [ ]:
ancho = 15
alto = 10
label_x = 'Fecha'
label_y = 'Clorofila (mg/m3)'
titulo = f'Resultados Experimento {exp} Prueba {prueba}'

file_svg_res = f'resultado_{exp}_{prueba}.svg'

file_png_res = f'resultado_{exp}_{prueba}.png'

# Preparación gráfica
fig, ax = plt.subplots(figsize=(ancho, alto))

# Crear el gráfico
plt.plot(dataset.index, dataset['chla'], label='Entrenamiento', color='royalblue')
plt.plot(df_best_tmsp.index, df_best_tmsp['feature'], label='Generados (reconstrucción)', linestyle='--', color='seagreen', alpha = 0.4)

# Configuración a los ejes / grid
    # Color de fondo
ax.set_facecolor('white')

    # Configuración del grid
ax.grid(True, color='lightgrey', linestyle='--', linewidth=0.7, axis='y', which='major')
ax.yaxis.grid(True, color='lightgrey', linestyle='--', linewidth=0.2, which='minor')

    # Configuración de los ticks del eje y
ax.yaxis.set_major_locator(ticker.MultipleLocator(10))   # Ticks mayores cada 1
ax.yaxis.set_minor_locator(ticker.MultipleLocator(50)) # Ticks menores cada 0.5

ax.tick_params(axis='y', labelsize=25)

    # Configuración de los ticks del eje x
ax.tick_params(axis='x', which='major', labelsize=25)

# Leyenda
ax.legend(fontsize=20)

# Configuración de las etiquetas
ax.set_xlabel(label_x, fontsize=30, labelpad=25)
ax.set_ylabel(label_y, fontsize=30, labelpad=25)
ax.set_title(titulo, fontsize=35, pad=10)

# Borrar líneas superior y derecha
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar imagen
plt.tight_layout()

# Guardar imagen
plt.savefig(file_svg_res, format='svg')
plt.savefig(file_png_res, format='png')

# Mostrar imagen
plt.show()

## Guardar resultados

In [ ]:
df_unido = pd.merge(df_best_tmsp, dataset, on='date', how='outer')

In [ ]:
df_unido

In [ ]:
df_unido['chla_full'] = df_unido['chla']
df_unido['chla_full'] = df_unido['chla_full'].fillna(df_unido['feature'])

In [ ]:
df_unido

In [ ]:
ancho = 15
alto = 10
label_x = 'Fecha'
label_y = 'Clorofila (mg/m3)'
titulo = f'Resultados Experimento {exp} Prueba {prueba}'

file_svg_unif = f'unificado_{exp}_{prueba}.svg'

file_png_unif = f'unificado_{exp}_{prueba}.png'

# Preparación gráfica
fig, ax = plt.subplots(figsize=(ancho, alto))

# Crear el gráfico
plt.plot(df_unido.index, df_unido['chla_full'], color='royalblue')

# Configuración a los ejes / grid
    # Color de fondo
ax.set_facecolor('white')

    # Configuración del grid
ax.grid(True, color='lightgrey', linestyle='--', linewidth=0.7, axis='y', which='major')
ax.yaxis.grid(True, color='lightgrey', linestyle='--', linewidth=0.2, which='minor')

    # Configuración de los ticks del eje y
ax.yaxis.set_major_locator(ticker.MultipleLocator(10))   # Ticks mayores cada 1
ax.yaxis.set_minor_locator(ticker.MultipleLocator(50)) # Ticks menores cada 0.5

ax.tick_params(axis='y', labelsize=25)

    # Configuración de los ticks del eje x
ax.tick_params(axis='x', which='major', labelsize=25)

# Leyenda
# ax.legend(fontsize=20)

# Configuración de las etiquetas
ax.set_xlabel(label_x, fontsize=30, labelpad=25)
ax.set_ylabel(label_y, fontsize=30, labelpad=25)
ax.set_title(titulo, fontsize=35, pad=10)

# Borrar líneas superior y derecha
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar imagen
plt.tight_layout()

# Guardar imagen
plt.savefig(file_svg_unif, format='svg')
plt.savefig(file_png_unif, format='png')

# Mostrar imagen
plt.show()

## Guardar resultados

In [ ]:
file_chla_data= f'chla_data_{exp}_{prueba}.csv'
df_unido.to_csv(file_chla_data, index=True)

In [ ]:
mejor_resultado

In [ ]:
conf_res_data = {
    'experimento': [exp],
    'prueba': [prueba],
    'timestep': [mejor_resultado[0]],
    'jensen_shannon': [mejor_resultado[1]],
    'wasserstein_distance': [mejor_resultado[2]],
    'kolmogorov_smirnov': [mejor_resultado[3]],
    'p_value': [mejor_resultado[4]]
}

conf_res= pd.DataFrame(conf_res_data)

file_conf_res = f'config_results_{exp}_{prueba}.csv'

conf_res.to_csv(file_conf_res, index=False)